In [1]:
import pandas as pd
df = pd.read_pickle('yonhap_ready.pkl')

In [2]:
df.head(3)

,article,title,article_len,sec1,sec_idx
0,'회장이 청부폭력' 루머 협박범도 기소…사이버명예훼손 전담수사 첫 사례 ...,"""해경이 가만있으라 방송"" 허위사실 유포 40대 기소",979,사회,4
1,지난 해 9월 북한의 함경북도 항구도시 나진과 러시아 극동지역 도시 ...,南 점검단 방북…나진-하산 프로젝트 시범운송 시작(종합),1550,정치,6
2,포항 도착한 화물선 러시아 시베리아산 유연탄 4만 500t을...,나진-하산 프로젝트 탄력…시범수송 석탄 포항 도착(종합),1827,경제,0


In [3]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(452922,)
(452922,)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(339691,)
(113231,)
(339691,)
(113231,)


In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 76.7 ms


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [6]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [7]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 21min 18s, sys: 13.7 s, total: 21min 32s
Wall time: 21min 32s


In [8]:
X_test_dtm = vect.transform(X_test)

In [9]:
X_train_dtm.shape

(339691, 9455550)

In [10]:
X_test_dtm.shape

(113231, 9455550)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [11]:
import pickle

with open("yonhap_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("yonhap_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [12]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [13]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 5.3 s, sys: 568 ms, total: 5.87 s
Wall time: 5.87 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [14]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [15]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.87247308599235196

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.92      0.91      0.91     28062
         과학       0.66      0.57      0.61       669
         국제       0.85      0.90      0.87     21955
         문화       0.75      0.91      0.82      6128
         사회       0.88      0.84      0.86     29722
         생활       0.55      0.76      0.64      2167
         정치       0.91      0.86      0.88     24528

avg / total       0.88      0.87      0.87    113231



In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [17]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [18]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 2min 13s, sys: 760 ms, total: 2min 14s
Wall time: 2min 14s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [19]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [20]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.90592682216000919

In [21]:
print(classification_report(y_test, y_pred_class_svm, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.94      0.94      0.94     28062
         과학       0.76      0.61      0.68       669
         국제       0.89      0.92      0.90     21955
         문화       0.87      0.87      0.87      6128
         사회       0.89      0.91      0.90     29722
         생활       0.81      0.60      0.69      2167
         정치       0.93      0.90      0.91     24528

avg / total       0.91      0.91      0.91    113231



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [24]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=4) & (y_test==4)])
print(np.argmax(counts))

3


In [25]:
X_test[(y_pred_class==3) & (y_test==4)]

282577               국내 유일의 독립애니메이션 영화제 '인디애니페스트 2016'이 ...
228246           세월호 추모앨범 'April' 낸 이지혜씨.  네티즌 300여 명 제작...
50793            '실크로드 경주 2015' 문화 향연       유라시아 문화특급을 주...
135662           곤지암리조트 스키장(연합뉴스 자료사진)        경기도 광주 곤지암...
42553                   전주국제영화제 사무처는 17일 기존 '전주 프로젝트: 삼인...
373212          마이크 걷어 내고 낮 공연 정례화…국·양악 넘나들며 다양한 실험  내달 ...
207900           김혁 선생의 초상화  만주 무장 독립운동 1세대 김혁 선생 손자 김진도...
376924       강원 삼척시민이 유럽에서 2018 평창동계올림픽 홍보활동에 나서 화제다.   ...
122270             국립장식미술관서 '코리아 나우' 전시 개막       루브르박물관 ...
378509           우리나라 최고의 명인·명창 등용문 가운데 하나인 춘향국악대전이 오는 5...
441264          광주 메이홀 다음 달 22일까지 시민 작품 접수   5·18 민주화운동을...
47896            대전시립미술관(연합뉴스 자료사진)        대전시립미술관은 2일부터...
29870              여의도 KBS본관 계단과 벽면 8천여장의 사연판으로 꾸며져     ...
66945                  ▲도서출판더알음(대표 서진수)은 빅데이터 활용과 관련해 대학...
41660            MBC 방송화면 캡처  뇌섹남·예지앞사·와리가리 등도 화제       ...
434171           전북 고창군이 출산율을 높이고 양성평등의식을 확산시키기 위해 진행한 '...
291960           '제3회 브런치북 프로젝트' 이벤트 안내 

In [26]:
df[df.index==421591]

,article,title,sec1,sec_idx
421591,최신 유행 트렌드 총집결 #흥(클릭!),"코미 ""FBI, 러시아 대선개입…트럼프팀과 공모 수사중""(속보)",국제,2


In [26]:
with open("yonhap_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [27]:
with open("yonhap_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)